In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

In [3]:
cd ..

C:\Projects\python\recommender


In [4]:
import torch as T
import torch.nn as nn
import torch.optim as optim

from datasets import MovelenDataset, TorchMovielen10k

## Init Dataloader instance

In [5]:
DEVICE = T.device('cpu')
BATCH = 32
SHUFFLE = False
WORKERS = 0
FILE_PATH = Path("./inputs/ml-100k/u.data")

In [6]:
databunch = TorchMovielen10k(FILE_PATH, user_min=4, item_min=4)
databunch.batch(BATCH)
databunch.device(DEVICE)
databunch.shuffle(SHUFFLE)
databunch.workers(WORKERS)

2019-09-04 15:39:59,832 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\ml-100k\u.data
I0904 15:39:59.832129  8652 torch_movielen.py:41] Read dataset in inputs\ml-100k\u.data
2019-09-04 15:39:59,843 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
I0904 15:39:59.843084  8652 torch_movielen.py:45] Original user size: 943
2019-09-04 15:39:59,846 - C:\Projects\python\recommender\utils.py - INFO - Original item size: 1682
I0904 15:39:59.846060  8652 torch_movielen.py:46] Original item size: 1682
2019-09-04 15:39:59,852 - C:\Projects\python\recommender\utils.py - INFO - Filter user size: 943
I0904 15:39:59.852046  8652 torch_movielen.py:52] Filter user size: 943
2019-09-04 15:39:59,855 - C:\Projects\python\recommender\utils.py - INFO - Filter item size: 1413
I0904 15:39:59.855036  8652 torch_movielen.py:53] Filter item size: 1413
c:\projects\python\recommender\.venv\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A 

In [7]:
train_dl = databunch.get_dataloader(dataset_type='train')
train_it = iter(train_dl)

In [8]:
user_index, pos_feats, neg_feats = train_it.next()
print("positive batch shape:", pos_feats.shape)
print("negtive batch shape:", neg_feats.shape)
print("user index shape:", user_index.shape)

positive batch shape: torch.Size([32, 3769])
negtive batch shape: torch.Size([32, 3769])
user index shape: torch.Size([32])


## Test each fm model

In [9]:
feat_dim = databunch.feat_dim
num_dim = 32
init_mean = 1

In [10]:
from models import TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM

### Test FM Model

In [11]:
fm_model = TorchFM(feat_dim, num_dim, init_mean)
fm_model

TorchFM()

In [12]:
user_index, pos_feats, neg_feats = train_it.next()

In [13]:
fm_model(pos_feats, neg_feats)

(tensor([-6.3320e+00, -1.8593e+01, -1.8187e+00, -3.6522e+00, -8.7877e+00,
         -1.4170e+01, -1.1217e+01,  3.2492e+01,  2.9107e+01, -1.5778e+00,
          2.8179e+01,  3.1098e+01,  4.7941e+00,  5.3306e+01, -1.4949e+01,
         -1.4957e+01,  8.3725e+01, -1.3907e+01, -1.0136e-02,  2.3218e+00,
         -2.1499e+01,  2.8128e+01, -1.1814e+00, -1.0797e+01, -2.7615e+01,
         -2.2710e+01,  1.6794e+01, -1.2926e+01,  2.9760e+01, -9.2205e+00,
         -1.1150e+01,  9.9238e+00], dtype=torch.float64,
        grad_fn=<SqueezeBackward0>),
 tensor([ 0.8997, -1.8188, -2.0658, -0.8265, -0.1294, -0.7602,  1.1092, -1.3965,
         -1.7074, -1.1005,  0.2752, -0.4844, -1.2156, -0.9740, -1.0221, -0.4601,
          0.7161, -1.1022, -0.6049, -0.6645, -1.3956,  0.6511, -0.6141,  0.7362,
         -1.0170, -1.1702, -1.4416, -0.6788, -0.8981, -0.5981, -0.0235, -0.6781],
        dtype=torch.float64, grad_fn=<SqueezeBackward0>))

### Test HRM FM model

In [14]:
hrm_model = TorchHrmFM(feat_dim, num_dim, init_mean)
hrm_model

TorchHrmFM()

In [15]:
hrm_model(pos_feats, neg_feats)

(tensor([33.1325, 31.2840, 33.8941, 37.8595, 24.3273, 33.6767, 33.3145, 24.9276,
         34.0461, 46.3949, 26.7207, 33.6015, 39.6543, 33.0167, 32.8380, 29.0679,
         32.3540, 32.9390, 37.6464, 36.5464, 27.9834, 36.6693, 29.5573, 32.4882,
         28.0673, 37.9960, 34.6276, 35.7300, 31.4886, 30.7740, 47.2476, 31.6606],
        dtype=torch.float64, grad_fn=<SqueezeBackward0>),
 tensor([37.1022, 32.9437, 27.1174, 30.4373, 29.8915, 27.6997, 33.6522, 30.2027,
         39.7062, 32.9901, 25.4198, 26.7987, 32.7458, 33.6718, 40.8421, 37.2893,
         31.7878, 35.4335, 33.6432, 31.2373, 23.8031, 40.4450, 29.9733, 37.3937,
         29.8326, 36.5847, 38.0195, 36.8252, 38.0957, 37.6037, 37.5518, 32.3737],
        dtype=torch.float64, grad_fn=<SqueezeBackward0>))

In [18]:
item = T.sparse.sum(pos_feats, dim=1)
item

tensor(indices=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29, 30, 31]]),
       values=tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
                      3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
                      3., 3., 3., 3.]),
       size=(32,), nnz=32, dtype=torch.float64, layout=torch.sparse_coo)

In [23]:
item.to_dense().unsqueeze(dim=1).shape

torch.Size([32, 1])

### Test PRME FM Model

In [24]:
prme_model = TorchPrmeFM(feat_dim, num_dim, init_mean)
prme_model

TorchPrmeFM()

In [25]:
prme_model(pos_feats, neg_feats)

(tensor([60.9180, 57.1923, 66.3343, 57.8448, 62.4383, 72.0836, 81.6217, 82.6911,
         67.9099, 69.5385, 63.9037, 59.3991, 58.1825, 63.3404, 61.0830, 58.7116,
         58.1612, 68.8459, 54.9932, 47.4228, 62.0930, 74.1544, 62.9308, 67.1919,
         66.9541, 65.5529, 54.0643, 62.9666, 57.2651, 66.6893, 57.8874, 67.4915],
        dtype=torch.float64, grad_fn=<SqueezeBackward0>),
 tensor([50.1787, 67.8435, 52.7835, 63.8109, 57.6609, 63.2887, 72.7375, 69.3349,
         73.9696, 63.7759, 52.7288, 67.4585, 61.2889, 73.3168, 61.4489, 55.8257,
         63.0113, 73.0728, 50.5091, 52.7749, 69.9864, 62.3278, 68.4112, 72.7717,
         45.5054, 60.6085, 60.6988, 64.1107, 67.2851, 60.9872, 54.5550, 64.5138],
        dtype=torch.float64, grad_fn=<SqueezeBackward0>))

### Test Trans FM Model

In [26]:
trans_model = TorchTransFM(feat_dim, num_dim, init_mean)

In [30]:
trans_model(pos_feats, neg_feats)

torch.Size([32, 1])
torch.Size([32, 1])


(tensor([[47.9871, 47.7071, 49.3781,  ..., 47.7364, 49.1120, 48.0285],
         [35.3914, 34.0698, 38.4322,  ..., 36.7168, 40.4203, 35.3162],
         [16.7107, 20.9923, 20.5644,  ..., 19.8710, 19.7025, 18.7586],
         ...,
         [44.4349, 44.1552, 43.0757,  ..., 43.8500, 45.3918, 46.0052],
         [38.0985, 35.4504, 36.1466,  ..., 38.6471, 38.6220, 35.9761],
         [40.7495, 37.8852, 37.1862,  ..., 38.5450, 41.7622, 39.7110]],
        dtype=torch.float64, grad_fn=<SqueezeBackward0>),
 tensor([ 92.4897,  95.9412,  71.5015,  83.8820,  94.2005, 110.1614,  88.4120,
          93.7598,  91.5013,  60.8054,  96.1686,  76.6246,  90.0194,  80.8533,
          79.8691,  88.4803, 108.2334, 110.7207, 107.8850, 101.6066,  76.7894,
         117.2790,  88.6113,  90.1018,  90.3195,  64.6207, 109.3250,  85.4523,
          76.7103,  90.7464,  85.9680,  82.5757], dtype=torch.float64,
        grad_fn=<SqueezeBackward0>))

#### Find size error!!

#### Test trans model forward

In [31]:
feature_dim = trans_model._feature_dim
param_linear = trans_model.param_linear
param_emb = trans_model.param_emb
param_trans = trans_model.param_trans

In [33]:
pos_linear = T.mm(pos_feats, param_linear)
neg_linear = T.mm(neg_feats, param_linear)
print(pos_linear.shape)
print(neg_linear.shape)

torch.Size([32, 1])
torch.Size([32, 1])


In [34]:
var_sum_op = T.ones(feature_dim, 1, dtype=T.double)
var_emb_product = T.sum(T.pow(param_emb, 2), dim=1, keepdim=True)
var_trans_product = T.sum(T.pow(param_trans, 2), dim=1, keepdim=True)
var_emb_trans_product = T.sum(param_emb * param_trans,
                              dim=1,
                              keepdim=True)
print(var_sum_op.shape)
print(var_emb_product.shape)
print(var_trans_product.shape)
print(var_emb_trans_product.shape)

torch.Size([3769, 1])
torch.Size([3769, 1])
torch.Size([3769, 1])
torch.Size([3769, 1])


In [35]:
# Common term positive
pos_feats_sum = T.mm(pos_feats, var_sum_op)
pos_emb_mul = T.mm(pos_feats, param_emb)
pos_trans_mul = T.mm(pos_feats, param_trans)

# Common terms negative
neg_feats_sum = T.mm(neg_feats, var_sum_op)
neg_emb_mul = T.mm(neg_feats, param_emb)
neg_trans_mul = T.mm(neg_feats, param_trans)

In [36]:
print(pos_feats_sum.shape)
print(pos_emb_mul.shape)
print(pos_trans_mul.shape)
print("---------------------------")
print(neg_feats_sum.shape)
print(neg_emb_mul.shape)
print(neg_trans_mul.shape)

torch.Size([32, 1])
torch.Size([32, 32])
torch.Size([32, 32])
---------------------------
torch.Size([32, 1])
torch.Size([32, 32])
torch.Size([32, 32])


In [37]:
# Term 1 pos
prod_term_pos = T.mm(pos_feats, var_emb_product)
term_1_pos = prod_term_pos * pos_feats_sum

# Term 1 neg
prod_term_neg = T.mm(neg_feats, var_emb_product)
term_1_neg = prod_term_neg * neg_feats_sum

In [38]:
print(prod_term_pos.shape)
print(term_1_pos.shape)
print("-----------------------")
print(prod_term_neg.shape)
print(term_1_neg.shape)

torch.Size([32, 1])
torch.Size([32, 1])
-----------------------
torch.Size([32, 1])
torch.Size([32, 1])


In [39]:
# Term 2 pos
prod_term_pos = T.mm(pos_feats, param_trans)
term_2_pos = prod_term_pos * pos_feats_sum

# Term 2 neg
prod_term_neg = T.mm(neg_feats, var_trans_product)
term_2_neg = prod_term_neg * pos_feats_sum

In [40]:
print(term_2_pos.shape)
print(term_2_neg.shape)

torch.Size([32, 32])
torch.Size([32, 1])


#### Find error !!!!

In [41]:
trans_model(pos_feats, neg_feats)

(tensor([ 93.3357,  87.8569,  76.6370,  86.5213,  80.0202, 133.7127,  94.9688,
          89.1366,  91.9335,  82.4871,  91.7615,  89.6770,  92.2411,  77.6657,
          87.7062,  98.8069,  94.2637, 108.8042,  98.6036, 101.5182,  97.6163,
          86.7221,  84.8828,  82.1383,  97.0879,  90.8993, 110.1206,  80.5946,
          98.5954,  94.4523,  84.5089,  94.7278], dtype=torch.float64,
        grad_fn=<SqueezeBackward0>),
 tensor([ 92.4897,  95.9412,  71.5015,  83.8820,  94.2005, 110.1614,  88.4120,
          93.7598,  91.5013,  60.8054,  96.1686,  76.6246,  90.0194,  80.8533,
          79.8691,  88.4803, 108.2334, 110.7207, 107.8850, 101.6066,  76.7894,
         117.2790,  88.6113,  90.1018,  90.3195,  64.6207, 109.3250,  85.4523,
          76.7103,  90.7464,  85.9680,  82.5757], dtype=torch.float64,
        grad_fn=<SqueezeBackward0>))